In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import functions as F

In [2]:
import os
import pandas as pd

# inputs
- year

In [3]:
from city_vars import dict_cities

In [10]:
city_proper = 'Austin'
year = '2015'
month = '5'
dict_city = dict_cities[city_proper]

In [5]:
gs_bkt = os.getenv('GCP_GCS_BUCKET')
jar_path = os.getenv('JAR_FILE_LOC')
creds_path = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

In [6]:
# additional ignore flag at end for cities with no data for year
conf = SparkConf() \
    .setMaster('spark://project-spark-1:7077') \
    .setAppName('project_year_read') \
    .set("spark.jars", jar_path)

### Only if an existing one already runs:
`sc.stop()`

In [ ]:
sc.stop()

In [7]:
sc = SparkContext(conf=conf)

22/11/03 10:22:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
hconf = sc._jsc.hadoopConfiguration()

hconf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hconf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hconf.set("fs.gs.auth.service.account.json.keyfile", creds_path)
hconf.set("fs.gs.auth.service.account.enable", "true")

In [9]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

***

### Replace below with me:
```
dict_df = {}
for city in cities:
    df_pq = spark.read \
        .option("header", "true") \
        .option(, ) \
        .parquet(f'{gcs_bkt}/pq/{city}/{year}/*')
    dict_df.update({city: df_pq})
```

In [11]:
df_pq = spark.read \
    .option("header", "true") \
    .parquet(f"{gs_bkt}/pq/from_raw/{dict_city['formatted']}/{year}/*")

### check count here: pq df
Command: `df_pq.count()`

Output: `485853`

In [ ]:
# filter out duplicates
# filter out rows with null values in important columns
# add timestamp column
df_time = df_pq \
    .distinct()

### check count here: non-null df
Command:
`df_time.count()`

Output:
`38114`

### check parsed dates

Command: `df_time.head(10)`

Output: must contain datetimes

### check date here: csv df, datetime col
Command:
```
df_time \
    .filter(F.month('timestamp') == 1) \
    .groupBy(F.dayofmonth('timestamp')) \
    .count() \
    .orderBy('count', ascending=False).show()
```
Output:
```
+---------------------+-----+
|dayofmonth(Timestamp)|count|
+---------------------+-----+
|                    1| 1825|
|                   12| 1353|
|                   15| 1312|
|                   13| 1311|
|                   26| 1296|
|                    6| 1290|
|                   17| 1288|
|                   18| 1278|
|                    5| 1267|
|                   20| 1256|
|                   16| 1251|
|                   23| 1250|
|                   10| 1237|
|                   11| 1228|
|                   27| 1215|
|                   19| 1214|
|                   30| 1211|
|                   31| 1189|
|                    9| 1184|
|                   25| 1183|
+---------------------+-----+
only showing top 20 rows
```

In [ ]:
parser_udf = F.udf(dict_city['parser'], returnType=dict_cities['p_ret_type'])

In [ ]:
# set up street column
# pick out important columns
df_select = df_clean \
    .select(dict_city['selected_cols']) \
    .withColumn(dict_city['p_new_col'], parser_udf(dict_city['p_orig_col'])) \
    .withColumn('city', F.lit(city_proper))

In [ ]:
df_select.write.parquet(f'{gs_bkt}/pq/clean/{dict_cities['formatted']}/{year}/{month}/', mode='overwrite')

In [ ]:
sc.stop()

### check date Jan: pq df
Command:
```
df_pq \
    .filter(F.month('Timestamp') == 1).count()
```
Output: `1825`

### check date Jan1: pq df
Command:
```
df_pq \
    .filter(F.dayofmonth('Timestamp') == 1).count()
```
Output: `1825`

### check date here: pq df
Command:
```
df_pq \
    .filter(F.month('Timestamp') == 1) \
    .groupBy(F.dayofmonth('Timestamp')) \
    .count() \
    .orderBy('count', ascending=False).show()
```